## Imports

In [ ]:
import pandas as pd
import numpy as np
import pypsa
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import plotly
import datetime
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from utils import market_values, market_values_by_time_index, nodal_balance, carrier_colors

In [ ]:
# dict(sorted(carrier_colors.items()))
# "aquamarine" in carrier_colors.values()

In [ ]:
n =pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")

## Case Study

In [ ]:
# example of bus and snapshot
bus = "DE0 1" # "AL0 0" # "DE0 1"
snap = "2013-01-01 09:00:00"
# energy carrier at bus
n.buses.carrier[bus]

In [ ]:
# active power at bus (positive if net generation at bus) in MW
n.buses_t.p.loc[snap,bus]

In [ ]:
# Lines (what reaches bus?)
# 1. sum of power @ lines with bus as bus0 and p0 (Active power at bus0 (positive if branch is withdrawing power from bus0)
n.lines_t.p0[n.lines[n.lines.bus0 == bus].index].sum(axis=1)
# 2. sum of power @ lines with bus as bus1 and p1 (Active power at bus1 (positive if branch is withdrawing power from bus1)
n.lines_t.p1[n.lines[n.lines.bus1 == bus].index].sum(axis=1)
# sum (if negative power is feed into bus)
lines = n.lines_t.p0[n.lines[n.lines.bus0 == bus].index].sum(axis=1) + n.lines_t.p1[n.lines[n.lines.bus1 == bus].index].sum(axis=1)
# sum should be the same value as the active power at the bus
lines[snap]

### Loads

In [ ]:
# active power at bus (positive if net load) in MW
load = n.loads_t.p.loc[snap,bus]

### Generators

In [ ]:
# generation of generators directly connected to bus
gens_index_direct = n.generators[n.generators.bus == bus].bus.index
# active power at bus (positive if net generation)
gen = n.generators_t.p.loc[snap, gens_index_direct].sum()

### Links

In [ ]:
# links connected to the bus (bus0 of link)
links_index_direct = n.links[n.links.bus0 == bus].index
# Active power at bus1 (positive if branch is withdrawing power from bus0).
n.links_t.p0.loc[snap, links_index_direct]#.sum()

In [ ]:
# links connected to the bus (bus1 of link)
links_index_direct = n.links[n.links.bus1 == bus].index
# Active power at bus1 (positive if branch is withdrawing power from bus0).
n.links_t.p1.loc[snap, links_index_direct]#.sum()

In [ ]:
# links connected to the bus (bus2 of link)
links_index_direct = n.links[n.links.bus2 == bus].index
# Active power at bus1 (positive if branch is withdrawing power from bus0).
n.links_t.p2.loc[snap, links_index_direct]

In [ ]:
# links connected to the bus (bus3 of link)
links_index_direct = n.links[n.links.bus3 == bus].index
# Active power at bus1 (positive if branch is withdrawing power from bus0).
n.links_t.p3.loc[snap, links_index_direct]

In [ ]:
# links connected to the bus (bus4 of link)
links_index_direct = n.links[n.links.bus4 == bus].index
# Active power at bus1 (positive if branch is withdrawing power from bus0).
n.links_t.p4.loc[snap, links_index_direct]

In [ ]:
# Overall balance from links
links_balance = 0

for i in range(0,5):
    links_index_direct = n.links[n.links[f"bus{i}"] == bus].index
    # Active power at busi (positive if branch is withdrawing power from busi).
    links_balance += n.links_t[f"p{i}"].loc[snap, links_index_direct].sum()

links_balance

### Storage Units

In [ ]:
# storage units
storage_index = n.storage_units[n.storage_units.bus == bus].index
# active power at bus (positive if net generation) in MW
su = n.storage_units_t.p.loc[snap, storage_index].sum()

### Stores

In [ ]:
# stores
stores_index = n.stores[n.stores.bus == bus].index
# active power at bus (positive if net generation) in MW
st = n.stores_t.p.loc[snap, stores_index].sum()

### Overall aggregation

In [ ]:
print(f"{gen} - {load} - {links_balance} + {su} + {st}")
gen - load - links_balance + su + st

In [ ]:
print(f"{gen} - {links_balance} + {su} + {st}")
gen  - links_balance + su + st

In [ ]:
# active power at bus (positive if net generation at bus) in MW
n.buses_t.p.loc[snap, bus]

Notes:
- shunt_impedances and transformers are not existent in network
- if you exclude the load in the balance, the bus is balanced. Why?

## Nodal Balance
- calculating import or export excess

### Electricity plot

In [ ]:
carrier = ["AC", "low voltage"]
loads = ["electricity", "industry electricity", "agriculture electricity"]
period = "2013-05"
nb_el = nodal_balance(n, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW and unstack
nb_el = nb_el.unstack(level=[1]) / 1000
load_el = nb_el[loads]
nb_el.drop
nb_el.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

df = nb_el

# split into df with positive and negative values and get colors
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
ax = df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename negative values so that they are not shown and plot negative values
ax = df_neg.rename(columns=lambda x: '_' + x).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# plot lmps
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier.isin(carrier)].index].mean(axis=1)[period]
ax2 = lmps.plot(style=":", color="black", label="lmp (mean over buses) [€/MWh]", secondary_y=True)
# set limits of secondary y-axis
ax2.set_ylim([ - 3 * lmps.max() * abs(df_neg.sum(axis=1).min()) /  df_pos.sum(axis=1).max() , 3 * lmps.max()])

# plot loads
# n.load_t.p_set (gesetzte energie nachfrage)
# loads = n.loads_t.p[n.buses[n.buses.carrier.isin(carrier)].index].sum(axis=1)[period] / 1000
# loads.plot(style="--", color="black", label="loads (sum over buses) [GWh]", secondary_y=True)

# rescale the y-axis
ax.set_ylim([df_neg.sum(axis=1).min(), df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.2, 1), title="Legend for left y-axis")
ax2.legend(title="Legend for right y-axis",  loc="upper center", bbox_to_anchor=(1.205, 0.43))
ax.set_ylabel("tota electriyity balance [GW]")
ax.set_xlabel("")
ax.set_title(f"{carrier}")
fig.tight_layout()

# februrary; GAS CHP

In [ ]:
n.loads.carrier.isin(["electricity", "industry electricity"])

### Electricity plot

In [ ]:
carrier = ["AC", "battery", "Li ion", "low voltage", "home battery"]
nb_el = nodal_balance(n, carrier = carrier, time="2013-05", aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW
nb_el = nb_el.unstack(level=[1]) / 1000
nb_el.head()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))

df = nb_el

# split into df with posititve and negative values and get colors
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename
df_neg.rename(columns=lambda x: '_' + x).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# rescale the y axis

ax.set_ylim([df_neg.sum(axis=1).min(), df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.17, 1))
ax.set_ylabel("tota electriyity balance [GW]")
ax.set_xlabel("")
ax.set_title(f"{carrier}")
fig.tight_layout()

### H2 plot

In [ ]:
carrier = ["H2"] # ["H2", "H2 liquid"]
nb_h2 = nodal_balance(n, carrier, time="2013-05", aggregate=['component', 'bus'], energy=True)   # in units of energy
# convert from MW to GW
nb_h2 = nb_h2.unstack(level=[1]) / 1000
nb_h2.head()

In [ ]:
# get carriers that are present
nb_h2.columns

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

df = nb_h2

# split into df with posititve and negative values and get colors
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename
df_neg.rename(columns=lambda x: '_' + x).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# rescale the y axis
ax.set_ylim([df_neg.sum(axis=1).min(), df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.1, 1))
ax.set_ylabel("total hydrogen balance [GW]")
ax.set_xlabel("")
ax.set_title(f"{carrier}")
fig.tight_layout()

### Heat plot

In [ ]:
nb_heat = nodal_balance(n, ["residential rural heat", "services rural heat", "residential urban decentral heat", "services urban decentral heat", "urban central heat"], time="2013-05", aggregate=['component', 'bus'], energy=True)   # in units of energy
# convert from MW to GW
nb_heat = nb_heat.unstack(level=[1]) / 1000
nb_heat.head()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))

df = nb_heat

# split into df with posititve and negative values and get colors
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename
df_neg.rename(columns=lambda x: '_' + x).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# rescale the y axis

ax.set_ylim([df_neg.sum(axis=1).min(), df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.21, 1.005))
ax.set_ylabel("total hydrogen balance [GW]")
ax.set_xlabel("")
ax.set_title(f"{carrier}")
fig.tight_layout()

In [ ]:
carrier = ["urban central heat"]
nb_ucheat = nodal_balance(n, carrier=carrier, time="2013-05", aggregate=['component', 'bus'], energy=True)   # in units of energy
# convert from MW to GW
nb_ucheat = nb_ucheat.unstack(level=[1]) / 1000
nb_ucheat.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

df = nb_ucheat

# split into df with posititve and negative values and get colors
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename
df_neg.rename(columns=lambda x: '_' + x).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# rescale the y axis

ax.set_ylim([df_neg.sum(axis=1).min(), df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.18, 1))
ax.set_ylabel("total energy balance [GW]")
ax.set_xlabel("")
ax.set_title(f"{carrier}")
fig.tight_layout()

## LMP Determination

In [ ]:
# find snap with easy determination of marginal generator
n.generators_t.p

In [ ]:
n.global_constraints

In [ ]:
plt.plot(n.buses_t.marginal_price[["AL0 0"]][0:100])

In [ ]:
# find snap with easy determination of marginal generator
n.generators_t.p

In [ ]:
n.buses_t.marginal_price

In [ ]:
n.generators.loc["AL0 0 solar"]